### Fake News Classifier Using Bidirectional RNN - LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

In [48]:
import pandas as pd

In [49]:
df=pd.read_csv('dataset/fake-news/train.csv')

In [50]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [51]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [52]:
###Drop Nan Values
df=df.dropna()

In [53]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [54]:
## Get the Dependent features
y=df['label']

In [55]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [56]:
X.shape

(18285, 4)

In [57]:
y.shape

(18285,)

In [58]:
import tensorflow as tf

In [59]:
tf.__version__

'2.3.0'

In [60]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [61]:
### Vocabulary size
voc_size=5000

#### Onehot Representation

In [62]:
messages = X.copy()

In [63]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [64]:
messages.reset_index(inplace = True)

In [65]:
import nltk
import re
from nltk.corpus import stopwords

In [66]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
len(messages)

18285

In [68]:
### Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0 ,len(messages)):
    review = re.sub('[^A-Za-z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
#print(review)

In [69]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [70]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[538, 3461, 4898, 1666, 1305, 1078, 2313, 235, 2390, 3323],
 [3924, 1884, 2386, 1920, 2798, 25, 4598],
 [577, 3678, 2416, 1781],
 [4578, 1566, 3354, 4899, 1898, 4294],
 [2287, 2798, 381, 3973, 2885, 3706, 2798, 945, 2505, 4357],
 [169,
  1875,
  1769,
  4206,
  1231,
  2468,
  3395,
  3220,
  3577,
  3184,
  4639,
  2932,
  2165,
  4184,
  4598],
 [467, 4211, 3705, 1110, 410, 3555, 4263, 876, 2129, 2593, 2539],
 [53, 574, 4626, 4173, 2214, 2504, 2468, 4006, 2129, 2593, 2539],
 [3246, 4541, 2213, 1616, 4890, 3140, 3428, 458, 2468, 4122],
 [3376, 4811, 890, 1306, 3418, 1907, 2399, 2467],
 [2502, 2482, 2887, 2887, 2860, 3736, 2165, 1284, 197, 4810, 296],
 [4899, 3612, 1305, 3140, 2468, 2214],
 [1692, 4754, 2233, 616, 1857, 4700, 4454, 2353, 1882],
 [2145, 117, 1597, 1572, 3895, 3664, 3624, 2129, 2593, 2539],
 [1325, 2295, 2433, 2481, 2778, 2129, 2593, 2539],
 [3953, 234, 2605, 161, 2998, 1597, 3886, 1806, 3165, 1247],
 [3010, 3087, 1884],
 [626, 831, 1979, 4971, 2468, 2521, 421, 4598],
 

### Embedding Representation

In [71]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)
print(embedded_docs)
len(embedded_docs)

[[   0    0    0 ...  235 2390 3323]
 [   0    0    0 ... 2798   25 4598]
 [   0    0    0 ... 3678 2416 1781]
 ...
 [   0    0    0 ... 2129 2593 2539]
 [   0    0    0 ... 4824 2711 2088]
 [   0    0    0 ... 2553  755 1154]]


18285

In [72]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  538,
       3461, 4898, 1666, 1305, 1078, 2313,  235, 2390, 3323])

In [73]:
## Creating model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [74]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [75]:
len(embedded_docs),y.shape

(18285, (18285,))

In [76]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [77]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [79]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 44ms/step - loss: 0.3041 - accuracy: 0.8532 - val_loss: 0.1948 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 7s 38ms/step - loss: 0.1369 - accuracy: 0.9467 - val_loss: 0.1980 - val_accuracy: 0.9234
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0934 - accuracy: 0.9627 - val_loss: 0.2582 - val_accuracy: 0.9100
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0638 - accuracy: 0.9766 - val_loss: 0.2826 - val_accuracy: 0.9180
Epoch 5/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0442 - accuracy: 0.9846 - val_loss: 0.3206 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 0.3968 - val_accuracy: 0.9138
Epoch 7/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0185 - accuracy: 0.9944 - val_loss: 0.4353 - val_accuracy: 0.9167

### Performance Metrics And Accuracy¶

In [80]:
y_pred=model.predict_classes(X_test)

In [81]:
from sklearn.metrics import confusion_matrix

In [82]:
confusion_matrix(y_test,y_pred)

array([[3111,  308],
       [ 195, 2421]], dtype=int64)

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9166528583264292

In [84]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3419
           1       0.89      0.93      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035

